In [1]:
using Revise
using HypergraphNB
using DataFrames
using Clustering
using Statistics
using RCall
using Arpack
using Plots
using LinearAlgebra
using StatsBase
using MultivariateStats
import Random

┌ Info: Precompiling HypergraphNB [2dbe4ab3-0c84-4f8d-86d2-f64d6d698dce]
└ @ Base loading.jl:1260
┌ Warning: Package HypergraphNB does not have StatsBase in its dependencies:
│ - If you have HypergraphNB checked out for development and have
│   added StatsBase as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with HypergraphNB
└ Loading StatsBase into HypergraphNB from project dependency, future warnings for HypergraphNB are suppressed.


In [127]:
N = [100, 100, 200]
C = [NaN, 5, 2, 5]
P = [NaN, 0.9, 0.1, 0.9]

z = vcat([repeat([z], N[z]) for z ∈ 1:length(N)]...)
n = sum(N)
ẑ = rand(1:length(N), n);

In [128]:
@time H = plantedPartitionHypergraph(N, C, P);

  0.008145 seconds (62.25 k allocations: 5.777 MiB)


In [129]:
@time B = reducedBPJacobian(H, ẑ);

  2.091005 seconds (74.48 k allocations: 33.164 MiB)


In [130]:
size(B)

(7200, 7200)

In [131]:
@time E = Arpack.eigs(B; nev = 5, ritzvec = true);            

  0.025648 seconds (296 allocations: 3.538 MiB)


In [144]:
V = hcat([HypergraphNB.transform_eigenvector(real.(E[2][:,i]), H) for i ∈ 1:length(N)]...)
V = real.(V)
V = V .> 0 # experiment
clus = Clustering.kmeans(V', length(N))
clusters = Clustering.assignments(clus);
randindex(clusters, z)[1]

0.528241428747899